In [93]:
#imports and definitions
import pandas as pd
import deeplabcut
import os
import numpy as np
def find_deeplabcut_project(_base_dir, _project_name, _experimenter_name):
    for folder in os.listdir(_base_dir):
        if folder.startswith(_project_name + "-" + _experimenter_name):
            return os.path.join(_base_dir, folder)
    return None

# Setup
To do analysis of data, create a new folder in the labeled-data directory, and paste your png files to analyze inside.

In [139]:
#setup: always run
project_name = "oscillator_test" #IMPORTANT: NO SPACES
experimenter_name = "Nils-Sommerfeld" #IMPORTANT: NO SPACES
training_folder_name = "test_train"
analysis_folder_name = "TEST_analyze"



base_directory = os.getcwd() #The new folder you created
config_path = find_deeplabcut_project(base_directory, project_name, experimenter_name) + "\\config.yaml"
print (config_path)
analysis_folder = find_deeplabcut_project(base_directory, project_name, experimenter_name) + "\\labeled-data\\" + analysis_folder_name
print(analysis_folder)
training_folder = find_deeplabcut_project(base_directory, project_name, experimenter_name) + "\\labeled-data\\" + training_folder_name
print(training_folder)

C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-22\config.yaml
C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-22\labeled-data\TEST_analyze
C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-22\labeled-data\test_train


# Do analysis

In [1]:
deeplabcut.analyze_images(config_path, [analysis_folder], save_as_csv=True)

# Convert analysis file

In [59]:
#Ok now we have to do something very stupid:
    #To refine labels on the analyzed data, we need to trick deeplabcut into thinking its training data.
    #This is harder than it seems, as training and analyzed data have different outputs.
file_to_convert = "";
for folder in os.listdir(analysis_folder):
    if folder.startswith("image_prediction") and folder.endswith(".h5"):
        file_to_convert = os.path.join(analysis_folder, folder)

#file_to_convert = "C:\\Users\\somme\\Documents\\DNA\\pose_test\\oscillator_test-Nils-Sommerfeld-2025-07-22\\labeled-data\\TEST_analyze\\image_predictions_DLC_Resnet50_oscillator_testJul22shuffle1_snapshot_090.h5"
print("File to convert: " + file_to_convert)

file_to_copy_format = training_folder + "\\CollectedData_" + experimenter_name + ".h5"
print("Training file to copy format from: " + file_to_copy_format)

File to convert: C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-22\labeled-data\TEST_analyze\image_predictions_DLC_Resnet50_oscillator_testJul22shuffle1_snapshot_090.h5
Training file to copy format from: C:\Users\somme\Documents\DNA\pose_test\oscillator_test-Nils-Sommerfeld-2025-07-22\labeled-data\test_train\CollectedData_Nils-Sommerfeld.h5


In [149]:
original_df = pd.read_hdf(file_to_copy_format)
column_format = original_df.columns  # a MultiIndex


data_to_copy = pd.read_hdf(file_to_convert)

images_list = [val for val in os.listdir(analysis_folder) if val.endswith(".png")]
length = len(images_list)
labeled_data_string_arr = ['labeled-data'] * length
analyzed_data_folder_name_string_arr = [analysis_folder_name] * length

multi_index = pd.MultiIndex.from_arrays([labeled_data_string_arr, analyzed_data_folder_name_string_arr, images_list],)

#CREATE NEW FILE IN CORRECT FORMAT
new_df = pd.DataFrame(columns=column_format, index=multi_index)
new_df[:] = np.nan



scorer = data_to_copy.columns.get_level_values(0).unique()[0]
bodyparts = data_to_copy.columns.get_level_values(2).unique()


#Copy data: i is a number: index of the image, j is a string of the current bodypart
for i in range(length):
    for j in bodyparts:
        for k in ["x", "y"]:
            new_df.loc[new_df.index[i], (experimenter_name, j, k)] = data_to_copy.loc[data_to_copy.index[i], (scorer, "animal", j, k)]
new_df.to_hdf("CollectedData_" + experimenter_name + ".h5", key="df_with_missing", mode="w")

#THEN PASTE THE CREATED FILE INTO YOUR ANALYSIS FOLDER

DLC_Resnet50_oscillator_testJul22shuffle1_snapshot_090
Index(['long_plat', 'long_hinge', 'long_arm', 'inner_arm_long', 'inner_hinge',
       'inner_arm_short', 'short_arm', 'short_hinge', 'short_plat'],
      dtype='object', name='bodyparts')
47.75517
nan
100


C:\Users\somme\AppData\Local\Temp\ipykernel_49020\1871266089.py:44: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->floating,key->block0_values] [items->MultiIndex([('Nils-Sommerfeld',       'long_plat', 'x'),
            ('Nils-Sommerfeld',       'long_plat', 'y'),
            ('Nils-Sommerfeld',      'long_hinge', 'x'),
            ('Nils-Sommerfeld',      'long_hinge', 'y'),
            ('Nils-Sommerfeld',        'long_arm', 'x'),
            ('Nils-Sommerfeld',        'long_arm', 'y'),
            ('Nils-Sommerfeld',  'inner_arm_long', 'x'),
            ('Nils-Sommerfeld',  'inner_arm_long', 'y'),
            ('Nils-Sommerfeld',     'inner_hinge', 'x'),
            ('Nils-Sommerfeld',     'inner_hinge', 'y'),
            ('Nils-Sommerfeld', 'inner_arm_short', 'x'),
            ('Nils-Sommerfeld', 'inner_arm_short', 'y'),
            ('Nils-Sommerfeld',       'short_arm', 'x'),
            (

# Refine frames
Paste the newly created .h5 file into your folder in labeled-data with your analyzed data, and then go back to the napari editor to see the labels

In [153]:
#Now run this again, and open your config file and the analysis folder. you should be able to see the predicted points of the network
#    and be able to edit them. And, when you save, a csv should be created that you can do analysis with.
deeplabcut.label_frames(config_path)